## Index Dividends
This workbook shows how to return information on the dividends of the constituents of a list of indices.

#### Learn more

To learn more about the Refinitiv LSEG Library for Python please join the Refinitiv Developer Community. By registering and logging into [LSEG's Developer Community Portal](https://developers.lseg.com/) you will have free access to a number of learning materials like
 [Overview](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python)
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/tutorials), 
 [Documentation](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on [Our Q&A Forum](https://community.developers.refinitiv.com/discussions). 
The LSEG's Developer Community is always be happy to help.

----

## Some Imports to start with

In [1]:
import datetime as dt
import time
from concurrent.futures import ThreadPoolExecutor
from threading import Lock
from typing import Union

import pandas as pd
from dateutil.relativedelta import relativedelta
import xlsxwriter

import lseg.data as ld
import lseg.data.content.ipa.financial_contracts as ldf
from lseg.data.content.ipa.financial_contracts import cross

pd.set_option('future.no_silent_downcasting', True)

## Open the data session

The open_session() function creates and open sessions based on the information contained in the lseg-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.
Here we're setting the request timeout to 5 minutes (300 seconds)

In [2]:
config = ld.get_config()
config.set_param("http.request-timeout", 300) # 5 min
ld.open_session()

<lseg.data.session.Definition object at 0x24f0c977850 {name='workspace'}>

## Some constants the script will work with

A constant is a special type of variable whose value cannot be changed.
Using a constant in a script improve its readability, its reusability and its maintainability

In [3]:
# Requested fields retrieved by the query
EX_DATE_FLD = "TR.DivExDate"
INDEX_CONSTITUENT_SHARES_FLD = "TR.IndexConstituentShares"
DIVIDEND_PAYMENT_TYPE_FLD = "TR.DivPaymentType"
INDEX_CONSTITUENT_RIC = "TR.IndexConstituentRIC"
INDEX_CURRENCY_FLD = "AVAIL(TR.CLOSEPRICE.CURRENCY,TR.INDEXCALCULATIONCURRENCY)"
INDEX_DIVISOR_FLD = "TR.IndexDivisor"
INDEX_NAME_FLD = "TR.IndexName"
CLOSE_PRICE_FLD = "TR.ClosePrice"
CLOSE_PRICE_FLD_CURR = "TR.ClosePrice(Curn={})"

# Some columns used below by the dataframe
INSTRUMENT_RIC_COL = "Instrument"
DIVIDEND_EX_DATE_COL = "Dividend Ex Date"
CONSTITUENT_RIC_COL = "Constituent RIC"
INDEX_CURRENCY_COL = "Calculation Currency"
INDEX_DIVISOR_COL = "Index Divisor"
OUTSTANDING_SHARES_COL = "Company Shares"
INDEX_SHARES_COL = "Index Shares"
DIVIDEND_AMOUNT_COL = "Adjusted Gross Dividend Amount"
DIVIDEND_PAYMENT_TYPE = "Dividend Payment Type"
INDEX_NAME_COL = "Index Name"
CLOSE_PRICE_COL = "Close Price"
RIC_COL = "RIC"
EX_DATE_COL = "Ex-Date"
DIVIDEND_COL = "Dividend"
CURRENCY_COL = "Currency"
DIVIDEND_CURRENCY_COL = "Dividend Currency"

# Renamed columns
DIVIDEND_INDEX_POINTS = "Div (Ind. Points)"
BENCHMARK_INDEX = "Benchmark Index"
BENCHMARK_INDEX_NAME = "Benchmark Index Name"
AGG_COL_1 = "No. of Company Xdiv"
AGG_COL_2 = "Div (Ind. Pts)"
DIVIDEND_IN_INDEX_CURRENCY = "Dividend in ({})"

# Fx Rates requested fields
FXCROSSCODE = "FxCrossCode"
FXCROSSSCALINGFACTOR = "FxCrossScalingFactor"
VALUATION_DATE = "ValuationDate"
FXSPOT = "FxSpot"
ERRORCODE = "ErrorCode"

## Some python functions the script will work with

A function is a block of code which only runs when it is called.
A function accepts data as parameter and can return data as result.

In [4]:
# Define a defensive get_data function to retrieve data from Workspace
def get_lseg_data(universe: Union[str, list], fields: Union[str, list], parameters: Union[str,dict] = {}) -> pd.DataFrame:    
    """Retrieves data from LSEG with retry logic.

    Args:
        universe: The universe of instruments to request data for.
        fields: The fields to retrieve.
        parameters: Optional parameters for the request.

    Returns:
        A pandas DataFrame containing the requested data.
    """
    res = pd.DataFrame()
    nb_relaunch_request = 5
    succeed = False
    time.sleep(1) # one second delay before a call to the back-end
    while not succeed and nb_relaunch_request > 0:
        try:
            res = ld.get_data(universe, fields, parameters)
        except Exception as ex:
            nb_relaunch_request -= 1
            time.sleep(0.2)
        else:
            succeed = True
    return res

# Returns a Dataframe which contain the errors occured during the execution of the script
def create_error_dataframe(index_ric, error_message):
    """Creates a DataFrame to store error information."""
    return pd.DataFrame({"Index": index_ric, "Error Message": error_message}, index = [0])

def _fetch_index_data(index_ric, start_time):
    """Fetch basic information about the index.
    Which are the Index RIC, the index name, the index currency, the CLose price of the index and the index divisor"""
    fields = [
        INDEX_CONSTITUENT_RIC,
        INDEX_NAME_FLD,
        INDEX_CURRENCY_FLD,
        CLOSE_PRICE_FLD,
        INDEX_DIVISOR_FLD,
        INDEX_CONSTITUENT_SHARES_FLD
    ]
    df = get_lseg_data(index_ric, fields)
    if df.empty:
        raise ValueError(f'No data returned for index {index_ric}. Please check your entitlement!\n'+
                        f'Elapsed time: {time.time() - start_time} seconds.')
    df.rename(columns = {INDEX_CURRENCY_FLD : INDEX_CURRENCY_COL}, inplace = True, errors = 'ignore')
    return df

def _fetch_constituent_dividends(index_ric, index_constituent_rics, index_currency, valuation_date_str, use_prev_fx_rate):
    """Fetches dividend data for the index contituents"""
    if use_prev_fx_rate:
        DIV_AMOUNT_CURR = "TR.DIVADJUSTEDGROSS"
        DIV_CURRENCY = "TR.DIVADJUSTEDGROSS.CURRENCY"
    else:
        DIV_AMOUNT_CURR = f"TR.DIVADJUSTEDGROSS(CURN={index_currency})"
        DIV_CURRENCY = f"TR.DIVADJUSTEDGROSS(CURN={index_currency}).CURRENCY"
    
    div_details_fields = [EX_DATE_FLD, DIV_AMOUNT_CURR, DIV_CURRENCY, DIVIDEND_PAYMENT_TYPE_FLD]
    # Request to return the Dividend Ex-Date, the adjusted gross dividend amount and the dividend payment type
    div_details = get_lseg_data(index_constituent_rics, div_details_fields, parameters = {"SDate": valuation_date_str})
    
    if len(div_details) == 0:
        raise ValueError(f'No data returned for the constituents of the index. Please check your entitlement!')
        
    div_details.rename(columns={
        INSTRUMENT_RIC_COL: RIC_COL,
        DIVIDEND_EX_DATE_COL: EX_DATE_COL,
        DIVIDEND_AMOUNT_COL: DIVIDEND_COL
    }, inplace = True, errors = 'ignore')
    return div_details

# Executes a query to retrieve the fx rates fx rates of a currencies pairs for a given market data date
def retrieve_fx_rates_data(fx_cross_codes, fx_rates, market_data_date):
    fx_spot_universe = []
    for cross_code in fx_cross_codes:
        fx_spot_universe.append(cross.Definition(fx_cross_type = cross.FxCrossType.FX_SPOT, fx_cross_code = cross_code))
    parameters = cross.PricingParameters(
        valuation_date = market_data_date.strftime("%Y-%m-%d"),
        price_side = cross.PriceSide.BID)
    output_fields = [FXCROSSCODE, VALUATION_DATE, FXSPOT, FXCROSSSCALINGFACTOR, ERRORCODE]
    ldf_def = ldf.Definitions(universe = fx_spot_universe, pricing_parameters = parameters, fields = output_fields).get_data()
    if ldf_def is not None:
        response_df = ldf_def.data.df
        response_len = len(response_df)
        if response_len > 0:
            for idx in range(response_len):
                error_code = response_df.loc[idx, ERRORCODE]
                fx_cross_code = response_df.loc[idx, FXCROSSCODE]
                fx_spot_rate = response_df.loc[idx, FXSPOT]
                fx_cross_scaling_factor = response_df.loc[idx, FXCROSSSCALINGFACTOR]
                if pd.isna(fx_cross_scaling_factor) or fx_cross_scaling_factor == 0.0:
                    fx_cross_scaling_factor = 1                
                if pd.isna(error_code) or len(error_code) == 0:
                    fx_rates[fx_cross_code] = fx_spot_rate / fx_cross_scaling_factor

    return fx_rates
    
# Return the fx rates of a currencies pairs for a given market data date
def get_fx_rates(fx_rates, index_curr, currencies, market_data_date):
    if len(currencies) == 0:
        return fx_rates
    
    fx_cross_codes = []
    for currency in currencies:
        if currency == index_curr or (currency + index_curr) in fx_rates:
            continue
        fx_cross_codes.append(currency + index_curr)
    
    default_fx_rates_key = index_curr + index_curr
    if default_fx_rates_key not in fx_rates:
        fx_rates[default_fx_rates_key] = 1.0

    return retrieve_fx_rates_data(fx_cross_codes, fx_rates, market_data_date) if len(fx_cross_codes) > 0 else fx_rates

# Applies the Fx Rates on the dividend value
def apply_fx_rates(div_value, div_currency, index_currency, fx_rates):
    if pd.isna(div_currency) or div_currency == '':
        return 0.0
    fx_cross_code = f"{div_currency}{index_currency}"
    fx_rate = fx_rates[fx_cross_code] if fx_cross_code in fx_rates else 0
    return div_value * fx_rate

#  Extract the FxRates for the export purpose
def extract_fx_rates(fx_rates, index_curr, currencies):
    output_dico = {}    
    for currency in currencies:
        if currency == index_curr:
            continue
        fx_cross_Code = f"{currency}{index_curr}"
        output_dico[fx_cross_Code] = fx_rates[fx_cross_Code]
    return output_dico
    
def _apply_fx_rates_to_dividends(div_details, index_currency, fx_rates, market_data_date, use_prev_fx_rate, locker):
    """Applies FX rates to dividends."""
    if use_prev_fx_rate:
        currency_pairs = [item for item in set(div_details[CURRENCY_COL]) if pd.isna(item) is False and item != '']
        fx_market_data_date = market_data_date + relativedelta(days=-1)
        fx_rates = get_fx_rates(fx_rates, index_currency, currency_pairs, fx_market_data_date)                    
        div_details[DIVIDEND_IN_INDEX_CURRENCY.format(index_currency)] = div_details.apply(
            lambda row: apply_fx_rates(row[DIVIDEND_COL], row[CURRENCY_COL], index_currency, fx_rates), axis = 1
        )
        with locker:
            fx_rates_dict = extract_fx_rates(fx_rates, index_currency, currency_pairs)
    else:                    
        div_details.rename(columns = {DIVIDEND_COL : DIVIDEND_IN_INDEX_CURRENCY.format(index_currency)}, inplace = True, errors = 'ignore')
        
    div_details.reset_index(inplace=True, drop=True) # [TO CHECK]
    return div_details, fx_rates_dict

def _calculate_index_divisor(index_ric, index_close_price, index_constituent_rics, index_currency, start_time, constituent_shares_df):
    """Calculates the index divisor if it's not provided."""
    close_price_details = get_lseg_data(index_constituent_rics, CLOSE_PRICE_FLD_CURR.format(index_currency))
    if len(close_price_details) > 0:
        close_price_details.rename(columns={INSTRUMENT_RIC_COL: RIC_COL}, inplace=True, errors='ignore')
        close_price_details = pd.merge(close_price_details, constituent_shares_df, how="inner", on=RIC_COL)
        close_price_details[CLOSE_PRICE_COL] = pd.to_numeric(close_price_details[CLOSE_PRICE_COL], errors='coerce').fillna(0)
        index_divisor = (close_price_details[OUTSTANDING_SHARES_COL] * close_price_details[CLOSE_PRICE_COL]).sum() / index_close_price
        return index_divisor
    else:
        raise ValueError(f'Internal error: Impossible to get Close price for the constituents of the index {index_ric}. The index divisor will not be computed!\n'+
                         f'Elapsed time: {time.time() - start_time} seconds.')
        error_details = create_error_dataframe(index_ric, error_message)
                    
# Returns the dividend details of the input index
def get_dividend_details(index_ric, use_prev_fx_rate, fx_rates, market_data_date, locker):
    """Calculates dividend details for a given index.

    Args:
        index_ric: The RIC of the index.
        use_prev_fx_rate: Whether to use previous day's FX rates.
        fx_rates: A dictionary to store and reuse FX rates.
        market_data_date: The market data date.
        locker: A lock for thread safety when updating shared fx_rates.

    Returns:
        A tuple containing:
            - index_info: DataFrame with index information.
            - div_details: DataFrame with dividend details.
            - error_details: DataFrame with any errors encountered.
            - fx_rates_dict: Dictionary with FX rates used.
    """
    # step 0) initialize variables
    start_time = time.time()
    print(f"Processing index '{index_ric}'...")
    
    index_info = pd.DataFrame(columns=[BENCHMARK_INDEX, BENCHMARK_INDEX_NAME, INDEX_DIVISOR_COL, CLOSE_PRICE_COL, CURRENCY_COL])
    div_details = pd.DataFrame()
    error_details = pd.DataFrame()
    fx_rates_dict = {}
    valuation_date_str = market_data_date.strftime("%Y-%m-%d")

    try:
        # step 1) fetch index data
        df = _fetch_index_data(index_ric, start_time)
        
        index_name = df.loc[0, INDEX_NAME_COL]
        index_divisor = df.loc[0, INDEX_DIVISOR_COL]
        index_close_price = df.loc[0, CLOSE_PRICE_COL]
        index_currency = df.loc[0, INDEX_CURRENCY_COL]
    
        if pd.isna(index_currency) or index_currency == "":
            index_currency = "USD"
        
        index_info.at[0, BENCHMARK_INDEX] = index_ric
        index_info.at[0, BENCHMARK_INDEX_NAME] = index_name
        index_info.at[0, INDEX_DIVISOR_COL] = index_divisor
        index_info.at[0, CLOSE_PRICE_COL] = index_close_price
        index_info.at[0, CURRENCY_COL] = index_currency

        df = df.dropna(subset=[CONSTITUENT_RIC_COL], axis=0)
        df[INDEX_SHARES_COL] = pd.to_numeric(df[INDEX_SHARES_COL], errors='coerce')
        df[INDEX_SHARES_COL] = df[INDEX_SHARES_COL].fillna(0)
        constituent_shares_df = df[[CONSTITUENT_RIC_COL, INDEX_SHARES_COL]].copy()
        constituent_shares_df.reset_index(inplace=True, drop=True)
        constituent_shares_df.rename(columns = {CONSTITUENT_RIC_COL : RIC_COL,  INDEX_SHARES_COL : OUTSTANDING_SHARES_COL}, inplace = True, errors = 'ignore')

        # step 2) fetch constituent dividends
        index_constituent_rics = list(df[CONSTITUENT_RIC_COL])
        
        div_details = _fetch_constituent_dividends(index_ric, index_constituent_rics, index_currency, valuation_date_str, use_prev_fx_rate)

        # step 3) process dividend data
        # Get only the latest dividend  amount and group them
        #   group by RIC, currency, dividend amount, payment type - then get max ex-date
        div_details = div_details.groupby([RIC_COL, CURRENCY_COL, DIVIDEND_COL, DIVIDEND_PAYMENT_TYPE])[EX_DATE_COL].max().reset_index()
        #   group by RIC, ex-date, currency, payment type - then sum dividend amount
        div_details = div_details.groupby([RIC_COL, EX_DATE_COL, CURRENCY_COL, DIVIDEND_PAYMENT_TYPE])[DIVIDEND_COL].sum().reset_index()

        # merge with constituent shares
        div_details = pd.merge(constituent_shares_df, div_details, how="inner", on=RIC_COL)

        # fill missing dividend and outstanding shares with 0
        div_details[[DIVIDEND_COL, OUTSTANDING_SHARES_COL]] = div_details[[DIVIDEND_COL, OUTSTANDING_SHARES_COL]].fillna(0)
        div_details.reset_index(inplace=True, drop=True)

        # step 4) apply fx rates and extract fx rates used
        div_details, fx_rates_dict = _apply_fx_rates_to_dividends(div_details, index_currency, fx_rates, market_data_date, use_prev_fx_rate, locker)

        # step 5) calculate index divisor (if needed)
        # If the index divisor is not provided, use the formula: index_divisor = (sum (outstanding_shares * stock_close_price)) / index_close_price
        if pd.isna(index_divisor) or index_divisor == "":
            index_divisor = _calculate_index_divisor(index_ric, index_close_price, index_constituent_rics, index_currency, start_time, constituent_shares_df)
            index_info.at[0, INDEX_DIVISOR_COL] = index_divisor
        if pd.isna(index_divisor) or index_divisor == "":
            div_details[DIVIDEND_INDEX_POINTS] = 0.0

        # step 6) calculate dividend impact
        div_details[DIVIDEND_INDEX_POINTS] = (div_details[DIVIDEND_IN_INDEX_CURRENCY.format(index_currency)] * div_details[OUTSTANDING_SHARES_COL]) / index_divisor
        div_details = div_details.sort_values(by=[EX_DATE_COL], ascending=True)

        # select relevant columns from div_details based on use_prev_fx_rate
        if use_prev_fx_rate:
            div_details = div_details[[RIC_COL, EX_DATE_COL, DIVIDEND_COL, CURRENCY_COL, DIVIDEND_IN_INDEX_CURRENCY.format(index_currency), OUTSTANDING_SHARES_COL, DIVIDEND_PAYMENT_TYPE, DIVIDEND_INDEX_POINTS]]
        else:
            div_details = div_details[[RIC_COL, EX_DATE_COL, DIVIDEND_IN_INDEX_CURRENCY.format(index_currency), OUTSTANDING_SHARES_COL, DIVIDEND_PAYMENT_TYPE, DIVIDEND_INDEX_POINTS]]

        print(f'Index {index_ric} processed! Elapsed time: {time.time() - start_time} seconds.')

    except ValueError as e:
        print(e)
        error_details = create_error_dataframe(index_ric, error_message)

    return index_info, div_details, error_details, fx_rates_dict

# Returns the dividend details of the list of indices
def get_dividends_details(index_rics_constituents, use_prev_fx_rate, valuation_date):
    """Calculates dividend details for multiple indexes.

    Args:
        index_rics_constituents: The RIC of the index.
        use_prev_fx_rate: Whether to use previous day's FX rates.
        valuation_date: Date of valuation.

    Returns:
        A tuple containing:
            - index_infos_lst: List with index information.
            - div_details_lst: List with dividend details.
            - errors_df: DataFrame with any errors encountered.
            - fx_rates_lst: List with FX rates used.
    """
    index_infos_lst = []
    div_details_lst = []
    err_details_lst = []
    fx_rates_lst = []
    fx_rates_dico = {}
    lock = Lock()

    with ThreadPoolExecutor(max_workers = None) as executor:
        nb_loop = len(index_rics_constituents)
        locker_lst = []
        for i in range (nb_loop):
            locker_lst.append(lock)
        for index_info, div_details, err_details, fx_rates_details in executor.map(get_dividend_details,
                                                                                   index_rics_constituents,
                                                                                   [use_prev_fx_rate] * nb_loop,
                                                                                   [fx_rates_dico] * nb_loop,
                                                                                   [valuation_date] * nb_loop,
                                                                                   locker_lst):
            index_infos_lst.append(index_info)
            div_details_lst.append(div_details)
            err_details_lst.append(err_details)
            fx_rates_lst.append(fx_rates_details)    

    errors_df =  pd.DataFrame()
    for err_df in err_details_lst:
        if len(err_df) > 0:
            errors_df = pd.concat([errors_df, err_df], ignore_index=True)

    return index_infos_lst, div_details_lst, errors_df, fx_rates_lst

### The list of indices

In [5]:
index_rics_constituents = ['.SPX', '.FTSE', '.N225']

In [6]:
print(index_rics_constituents)

['.SPX', '.FTSE', '.N225']


### Processing the list of indices...

In [7]:
index_info_lst = []
div_details_lst = []
error_details_df = []
fx_rates_lst = []

today = dt.datetime.today()
start_date = dt.datetime(today.year, today.month, today.day)
end_date = start_date + relativedelta(years=1)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')
use_previous_fx_rate = True

if __name__ == '__main__':
    # Return a  dataframe which contain the dividend details
    index_info_lst, div_details_lst, error_details_df, fx_rates_lst = get_dividends_details(index_rics_constituents, use_previous_fx_rate, today)

ld.close_session()

Processing index '.SPX'...
Processing index '.FTSE'...
Processing index '.N225'...
Index .N225 processed! Elapsed time: 16.14514136314392 seconds.


C:\Users\U6086063\Miniconda3\envs\ldlib-24\lib\site-packages\pandas\core\dtypes\cast.py:1057:RuntimeWarning: invalid value encountered in cast
C:\Users\U6086063\Miniconda3\envs\ldlib-24\lib\site-packages\pandas\core\dtypes\cast.py:1081:RuntimeWarning: invalid value encountered in cast


Index .SPX processed! Elapsed time: 34.40031361579895 seconds.
Index .FTSE processed! Elapsed time: 35.474366426467896 seconds.


### Print result dataframe

In [11]:
for index_info in index_info_lst:
    display(index_info)
for div_details in div_details_lst:
    display(div_details.head(3))
display(error_details_df)
print(fx_rates_lst)

,Benchmark Index,Benchmark Index Name,Index Divisor,Close Price,Currency
0,.SPX,S&P 500,8295382858.0589,5970.84,USD


,Benchmark Index,Benchmark Index Name,Index Divisor,Close Price,Currency
0,.FTSE,FTSE 100,246606398,8149.78,GBP


,Benchmark Index,Benchmark Index Name,Index Divisor,Close Price,Currency
0,.N225,Nikkei 225 Index,30.819691,40281.16,JPY


,RIC,Ex-Date,Dividend,Currency,Dividend in (USD),Company Shares,Dividend Payment Type,Div (Ind. Points)
322,MNST.OQ,1990-11-08,0.005208,USD,0.005208,700214400,Cash Dividend,0.00044
21,AMD.OQ,1995-04-27,0.005,USD,0.005000,1622807000,Cash Dividend,0.000978
78,UAL.OQ,2008-01-07,2.15,USD,2.150000,328876000,Cash Dividend,0.085238


,RIC,Ex-Date,Dividend,Currency,Dividend in (GBP),Company Shares,Dividend Payment Type,Div (Ind. Points)
94,ULVR.L,2024-02-22,0.3647,GBP,0.3647,2463527739.99999,Cash Dividend,3.643249
70,RIO.L,2024-03-07,2.0377,GBP,2.0377,1068562153.0,Cash Dividend,8.829492
55,BEZG.L,2024-03-21,0.142,GBP,0.1420,637846624.374162,Cash Dividend,0.367283


,RIC,Ex-Date,Dividend,Currency,Dividend in (JPY),Company Shares,Dividend Payment Type,Div (Ind. Points)
134,2282.T,2024-03-28,119.0,JPY,119.00,0.5,Cash Dividend,1.930584
168,4578.T,2024-06-27,60.0,JPY,60.00,1.0,Cash Dividend,1.946807
39,4324.T,2024-06-27,69.75,JPY,69.75,1.0,Cash Dividend,2.263163


""


[{}, {'USDGBP': 0.794786202511524, 'EURGBP': 0.828723573358766}, {}]


### Export to excel

Export the result to excel for future usage.

In [9]:
# Writes a well formatted Excel sheet
def write_sheet(wb, index_info_df, div_summary_df, div_details_item, sheetname, title, fx_rates_info):
    # constants from excel fomatting
    BLUE_COLOR_LSEG = "#001EFF"
    WHITE_COLOR_PENCIL = "#FFFFFF"
    BLACK_BACKGROUND_COLOR = "#3C3C3C"
    GRAY_BACKGROUND_COLOR = "#E2E2E2"
    FONT_NAME = "Arial"
    FONT_SIZE_TITLE = 17
    FONT_SIZE_HEADER = 11
    FONT_SIZE_DETAILS = 10
    DATE_TIME_FORMAT = "yyyy-mm-dd"

    startrow_data_initial = 3
    startcol_data_initial = 1
    startrow_data = startrow_data_initial
    startcol_data = startcol_data_initial

    # Create the worksheet
    worksheet = wb.add_worksheet(sheetname)

    nb_columns = len (div_details_item.columns)

    # Set the size of the columns
    longuest_header = len(max(index_info_df.columns.values, key = len)) + 2

    # Set the size of the columns
    worksheet.set_column(1, nb_columns, longuest_header)

    # Do not show grid lines
    worksheet.hide_gridlines(2)

    # Format of a row of data
    text_format = wb.add_format({"border": 1, "font_name": FONT_NAME, "font_size": FONT_SIZE_DETAILS})

    # Format for a cell which is a datetime
    datetime_format = wb.add_format({"border": 1, "font_name": FONT_NAME, "font_size": FONT_SIZE_DETAILS, "num_format": DATE_TIME_FORMAT})

    # Format of the index info header
    index_info_format = wb.add_format({"font_name": FONT_NAME,
                                       "bold": True,
                                       "text_wrap": True,
                                       "fg_color": GRAY_BACKGROUND_COLOR,
                                       "font_size": FONT_SIZE_HEADER,
                                       "border": 1})
    # Format of the title
    title_format = wb.add_format({"font_name": FONT_NAME,
                                  "bold": True,
                                  "align": "center",
                                  "color": WHITE_COLOR_PENCIL,
                                  "fg_color" : BLUE_COLOR_LSEG,
                                  "size": FONT_SIZE_TITLE})

    # Format the column header of the dataframe
    header_format = wb.add_format({"font_name": FONT_NAME,
                                   "bold": False,
                                   "text_wrap": True,
                                   "align": "center",
                                   "color": WHITE_COLOR_PENCIL,
                                   "fg_color": BLACK_BACKGROUND_COLOR,
                                   "font_size": FONT_SIZE_HEADER,
                                   "border": 1})

    # Add the title in some merged cells
    worksheet_title = title + " on " + dt.datetime.now().strftime("%d %b %Y")
    worksheet.merge_range(0, startcol_data, 0, nb_columns, worksheet_title, title_format)

    # Add the information details    
    for col_num, value in enumerate(index_info_df.columns.values):
        worksheet.merge_range(startrow_data, startcol_data, startrow_data, startcol_data + 1, value, index_info_format)
        startrow_data = startrow_data + 1

    # Write the info details
    startrow_data = startrow_data_initial
    _, col_idx = index_info_df.shape
    for c in range(col_idx):        
        worksheet.write(startrow_data, startcol_data + 2, index_info_df.values[0, c], text_format)
        startrow_data = startrow_data + 1

    # dividends header details
    for col_num, value in enumerate(div_details_item.columns.values):
        worksheet.write(startrow_data, col_num + startcol_data, value, header_format)

    # dividends details
    row_idx, col_idx = div_details_item.shape
    startrow_data = startrow_data + 1
    for r in range(row_idx):
        for c in range(col_idx):
            value_to_write = div_details_item.values[r, c]
            cell_format = datetime_format if isinstance(value_to_write, dt.datetime) else text_format
            worksheet.write(r + startrow_data, c + startcol_data, value_to_write, cell_format)
    
    # Fx Rates info
    if len(fx_rates_info) > 0:
        startrow_data = startrow_data_initial
        startcol_data = startcol_data_initial + len(div_details_item.columns) + 2
        worksheet.write(startrow_data, startcol_data, "FxRate", header_format)
        for col_num, fx_cross_code in enumerate(fx_rates_info):
            worksheet.write(startrow_data, col_num + startcol_data + 1, fx_cross_code, header_format)
            worksheet.write(startrow_data + 1, col_num + startcol_data + 1, fx_rates_info[fx_cross_code], text_format)

    # dividend summary header
    startrow_data = startrow_data_initial + len(index_info_df.columns)
    startcol_data = startcol_data_initial + len(div_details_item.columns) + 2
    worksheet.write(startrow_data, startcol_data, div_summary_df.index.name, header_format)
    for col_num, value in enumerate(div_summary_df.columns.values):
        worksheet.write(startrow_data, col_num + startcol_data + 1, value, header_format)

    # Dividend summary details:
    row_idx, col_idx = div_summary_df.shape
    startrow_data = startrow_data + 1
    for r in range(row_idx):
        worksheet.write(r + startrow_data, startcol_data, div_summary_df.index[r], datetime_format)
        for c in range(col_idx):
            worksheet.write(r + startrow_data, c + startcol_data + 1, div_summary_df.values[r, c], text_format)

    # Display a chart
    chart = workbook.add_chart({'type': 'column'})    
    endrow_data = startrow_data + len(div_summary_df) - 1
    chart.add_series({
        "Name" : "My Sample Plot",
        "categories": [sheetname, startrow_data, startcol_data, endrow_data, startcol_data],
        "values": [sheetname, startrow_data, startcol_data + 1, endrow_data, startcol_data + 1]
    })
    chart.set_title({"name" : title})
    chart.set_legend({'position': 'none'})
    worksheet.insert_chart(endrow_data + 3, startcol_data, chart, {"x_scale": 2, "y_scale": 1})

    # Set the column size for the dividend summary details
    longuest_header = len(max(div_summary_df.columns.values, key = len)) + 1

    # Set the size of the columns
    len_misc = max(len(div_summary_df.columns), len(fx_rates_info))
    worksheet.set_column(startcol_data, startcol_data + len_misc, longuest_header)


In [10]:
print(f"Process finished! Exporting to Excel...!")
    
# Export to Excel
EXCEL_FILENAME_XLS = f"Index_Dividends_{start_date_str}.xlsx"
EXCEL_ERROR_FILENAME_XLS = f"Index_Dividends_Errors_{start_date_str}.xlsx"

# Create a workbook
workbook = xlsxwriter.Workbook(EXCEL_FILENAME_XLS, {"nan_inf_to_errors": True, "strings_to_numbers": True})

for idx in range(len(div_details_lst)):

    div_details_item = div_details_lst[idx]
    if len(div_details_item) == 0:
        continue

    # Filter by date
    div_summary_df = div_details_item[div_details_item[EX_DATE_COL] >= start_date_str]
    div_summary_df = div_summary_df[div_summary_df[EX_DATE_COL] <= end_date_str]

    # Group the dividend value by date
    div_summary_df = div_summary_df.groupby([EX_DATE_COL])[DIVIDEND_INDEX_POINTS].agg(['sum','count'])
    div_summary_df.rename(columns = {'count' : AGG_COL_1, 'sum' : AGG_COL_2}, inplace = True)

    index_info_df = index_info_lst[idx]
    sheet_name_label = index_info_df.loc[0, BENCHMARK_INDEX]
    sheet_title = f"{sheet_name_label} - Dividend Index Points"

    # Get the Fx Rates
    fx_rates_info = fx_rates_lst[idx]

    div_details_item.replace({pd.NaT: None}, inplace=True)
    index_info_df.replace({pd.NaT: None}, inplace=True)
    write_sheet(workbook, index_info_df, div_summary_df, div_details_item, sheet_name_label, sheet_title, fx_rates_info)


workbook.close()

if(not error_details_df.empty):
    # Write the error file
    error_details_df.to_excel(EXCEL_ERROR_FILENAME_XLS, index = False)

print(f"Process finished!")

Process finished! Exporting to Excel...!
Process finished!
